In [1]:
import argparse
from pathlib import Path
import glob
import os

## import the tools
from pathlib import Path
import os
import numpy as np
import pandas as pd
import laspy

import torch
## to find the neighbor points prediction
from sklearn.neighbors import KDTree
import numpy as np


## import the model tools
from torch_geometric.transforms import Compose
from torch_points3d.core.data_transform import MinPoints, XYZFeature, AddFeatsByKeys, GridSampling3D
from torch_points3d.core.data_transform.features import AddOnes
from torch_points3d.applications.pretrained_api import PretainedRegistry
from torch_geometric.data import Batch
from torch_points3d.metrics.confusion_matrix import ConfusionMatrix


In [2]:
def get_nearest_neighbors(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    tree = KDTree(candidates, leaf_size=20, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    closest = np.squeeze(indices)
    return closest



In [3]:
def load_model(model_path: str, data_path: str):
    model = torch.load(model_path)
    model['run_config']['data']['dataroot'] = data_path
    # print(model['run_config']["data"])
    # model['run_config']['data']['path_to_model'] = "/home/jf/Documents/msc/torch-3dpoints-powerline/models/preprocess_CNN/cnnStateDict.pth"
    torch.save(model, model_path)
    # # transformer for non ones
    # pos_z = [ "pos_z" ]
    # list_add_to_x = [ True ]
    # delete_feats = [ True ]
    # first_subsampling = model['run_config']["data"]["first_subsampling"]
    # transform_test = Compose([MinPoints(512),
    #                     XYZFeature(add_x=False, add_y=False, add_z= True),
    #                     AddFeatsByKeys(list_add_to_x=list_add_to_x, feat_names= pos_z,delete_feats=delete_feats),
    #                     GridSampling3D(mode='last', size=first_subsampling, quantize_coords=True)
    #                     ])

    # transformer for ones
    pos_z = [ "ones" ]
    list_add_to_x = [ True ]
    delete_feats = [ True ]
    first_subsampling = model['run_config']["data"]["first_subsampling"]
    input_nc_feats = [1]

    transform_test = Compose([MinPoints(512),
                     AddOnes(),
                     AddFeatsByKeys(list_add_to_x=list_add_to_x, feat_names= pos_z,delete_feats=delete_feats, input_nc_feats=input_nc_feats),
                     GridSampling3D(mode='last', size=first_subsampling, quantize_coords=True)
                     ])
    ### ['latest', 'loss_seg', 'acc', 'macc', 'miou']
    model_pl = PretainedRegistry.from_file(model_path, weight_name="miou").cuda()
    return model_pl, transform_test, model['run_config']['data']


In [4]:
def predict(room_info, model, filename, transform_test, predict_folder):
    ## loop for every files
    room_coord_mins = room_info['room_coord_min']
    room_coord_scales = room_info['room_coord_scale']
    files = list(glob.glob(predict_folder + f"/*{filename}*cloud*pt"))

    pred_data = []

    for file in files:
        sample = os.path.join(predict_folder, file)
        pt_data = torch.load(sample)
        room_index = pt_data['room_idx']

        room_coord_scale = room_coord_scales[room_index]
        pos_ = pt_data['points']
        point_in_original_las = pos_ * room_coord_scale + room_coord_mins[room_index]

        data_s = transform_test(Batch(pos=torch.from_numpy(pos_).float()))
        data_s.batch = torch.zeros(len(data_s.pos))
        data_s.y = torch.zeros(data_s.pos.shape[0]).long()
        index_to_nearst_neighbor = get_nearest_neighbors(pos_, data_s.pos)


        with torch.no_grad():
            model.eval()
            model.set_input(data_s, "cuda")
            model.forward(data_s)
        
        pre = model.output.cpu().numpy()
        m = torch.nn.functional.softmax(torch.tensor(pre), dim=1)
        cla_pre = np.argmax(m, axis=1)
        pre_ori = np.arange(len(pos_))
        if len(pos_) == 1:
            pre_ori[0] = cla_pre[0]
        else:
            for i in pre_ori:
                pre_ori[i] = cla_pre[index_to_nearst_neighbor[i]]
        combine_pre = np.column_stack((point_in_original_las, pre_ori.T))

        pred_data.append(combine_pre)

    pred_data = np.array([item for sublist in pred_data for item in sublist])

    return pred_data

In [5]:
def get_metrics(pred_data, las_file, whole_las):
    ## read original las file

    powerline_pts = pred_data[np.where(pred_data[:,3] == 1)].copy()
    powerline_pts_coord = powerline_pts[:,:-1].astype(np.int32) 
    # print(len(pred_data))
    pred_data = np.unique(pred_data, axis=0)
    # print(len(pred_data))

    las_file_point_data = np.stack([las_file.X, las_file.Y, las_file.Z], axis=0).transpose((1, 0))
    las_file_idx = get_nearest_neighbors(powerline_pts_coord, las_file_point_data)
    pred = np.zeros(len(las_file_point_data))
    las_file_label = las_file.classification 
    las_file_label = las_file_label==14 
    pred[las_file_idx] = 1
    pred = np.asarray(pred,dtype=np.int16)
    las_file_label = np.asarray(las_file_label,dtype=np.int16)
    # print(len(las_file_point_data))

    cfm = ConfusionMatrix()
    #Use as count_predicted_batch(true pred)
    cfm.count_predicted_batch(las_file_label, pred)
    las_file_metric_dict = {}

    las_file_metric_dict["acc"] = 100 * cfm.get_overall_accuracy()
    las_file_metric_dict["macc"] = 100 * cfm.get_mean_class_accuracy()
    las_file_metric_dict["miou"] = 100 * cfm.get_average_intersection_union()
    las_file_metric_dict["miou_class"] = {
        i: "{:.2f}".format(100 * v)
        for i, v in enumerate(cfm.get_intersection_union_per_class()[0])
    }
    las_file_metric_dict["precision"] = cfm.get_confusion_matrix()[1][1]/(cfm.get_confusion_matrix()[1][1]+cfm.get_confusion_matrix()[0][1])
    las_file_metric_dict["recall"] = cfm.get_confusion_matrix()[1][1]/(cfm.get_confusion_matrix()[1][1]+cfm.get_confusion_matrix()[1][0])
    las_file_metric_dict["cfm"] = cfm.get_confusion_matrix()


    whole_las_metric_dict = {}

    return las_file_metric_dict, whole_las_metric_dict


In [6]:
data_path = "/home/jf/data"
raw_data_path = "/home/jf/data/denmark/raw/"
root_path = "/home/jf/Documents/msc/torch-3dpoints-powerline/"
model_folder = "2023-05-22/20-40-06/"
model_name = "SEUNet18.pt"
model_path = root_path+ "outputs/" + model_folder + model_name

model, transform_test, config = load_model(model_path, data_path)

## load transform pt pre
processed_folder_name = config["processed_folder"] 
data_root_path = os.path.join(config['dataroot'] , "denmark")
processed_data_root_path = os.path.join(data_root_path, processed_folder_name)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
loading processed train split
Total of 14594 samples in train set.
loading processed val split
Total of 815 samples in val set.
loading processed test split
Total of 1917 samples in test set.
False
64
1
343
False
64
64
27
False
64
64
27
False
64
64
27
False
64
64
27
False
128
64
1
False
64
64
8
False
128
64
27
False
128
128
27
False
128
128
27
False
128
128
27
False
256
128
1
False
128
128
8
False
256
128
27
False
256
256
27
False
256
256
27
False
256
256
27
False
512
256
1
False
256
256
8
False
512
256
27
False
512
512
27
False
512
512
27
False
512
512
27


INFO - 2023-05-29 00:29:27,989 - model_checkpoint - Available weights : ['latest', 'loss_seg', 'acc', 'macc', 'miou', 'precision', 'recall']
INFO - 2023-05-29 00:29:27,990 - model_checkpoint - Model loaded from SEUNet18.pt:best_miou.


True
256
512
8
False
256
512
1
False
256
512
27
False
256
256
27
False
256
256
27
False
256
256
27
True
128
256
8
False
128
256
1
False
128
256
27
False
128
128
27
False
128
128
27
False
128
128
27
True
64
128
8
False
64
128
1
False
64
128
27
False
64
64
27
False
64
64
27
False
64
64
27
True
32
64
8
False
64
96
1
False
64
96
27
False
64
64
27
False
2
64
1


In [7]:
splits = ["test", "val", "test"]
split_dict = {}
for split in splits[:1]:
    metric_dict = {}
    if split == "train":
        overlap = config["train_overlap"]
    if split == "val":
        overlap = 0
    if split == "test":
        overlap = 0
    predict_folder_name = f"{split}_{overlap}_({config['block_size_x']}, {config['block_size_y']})"
    predict_folder = os.path.join(processed_data_root_path, predict_folder_name)
    pre_trans_path = os.path.join(predict_folder, "stats.pt")
    room_info = torch.load(pre_trans_path)
    
    files = room_info['room_names']
    for filename in files:
        print(f"Predecting on {filename}")
        #the unprocessed file
        # raw_file_path = os.path.join(raw_data_path,split,filename+".laz")
        # raw_file = laspy.read(raw_file_path, laz_backend=laspy.compression.LazBackend.LazrsParallel)
        #the file that the models sees
        model_file_path = os.path.join(raw_data_path,split,"NewLaz",filename+".laz")
        model_file = laspy.read(model_file_path, laz_backend=laspy.compression.LazBackend.LazrsParallel)

        pred_data = predict(room_info, model, filename, transform_test, predict_folder)
        model_las_metric_dict, whole_las_metric_dict = get_metrics(pred_data, model_file,"")#raw_file)
        metric_dict[filename] = {"model":model_las_metric_dict, "whole":whole_las_metric_dict}
    
    split_dict[split] = metric_dict

Predecting on PUNKTSKY_00004_1km_6106_492
4095
Predecting on PUNKTSKY_00005_1km_6219_494
4095
Predecting on PUNKTSKY_00004_1km_6106_510
4095
Predecting on PUNKTSKY_00004_1km_6106_494
4095
Predecting on PUNKTSKY_00005_1km_6211_474
4095
Predecting on PUNKTSKY_00004_1km_6105_518
4095
Predecting on PUNKTSKY_00004_1km_6106_493
4095


In [8]:
from collections import defaultdict
#metric_dict = split_dict["train"]
mean_metric_dict = {}
for key, metric_dict in split_dict.items():
    split_model_dict = defaultdict(list)
    split_whole_dict = defaultdict(list)
    for filename, seen_metric_dicts in metric_dict.items():
        # print(filename)
        # print(seen_metric_dicts)
        for split, file_metric_dict in seen_metric_dicts.items():
            for metric_name, metric_value in file_metric_dict.items():
                if split == "model":
                    if metric_name != "miou_class":
                        split_model_dict[metric_name].append(metric_value)
                    else:
                        for _class, value in metric_value.items():
                            split_model_dict[f"miou_{_class}"].append(float(value))
                else:
                    if metric_name != "miou_class":
                        split_whole_dict[metric_name].append(metric_value)
                    else:
                        for _class, value in metric_value.items():
                            split_whole_dict[f"miou_{_class}"].append(float(value))

    mean_dict = {}
    for metric_key, metric_values in split_model_dict.items():
        if metric_key!="cfm":
            mean = sum(metric_values) / len(metric_values)
        else:
            mean = np.add.reduce(metric_values)

        mean_dict[metric_key] = mean
    mean_metric_dict[key] = mean_dict

In [9]:
import json
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [10]:
print(json.dumps(metric_dict,sort_keys=True, indent=4,cls=NumpyEncoder))

{
    "PUNKTSKY_00004_1km_6105_518": {
        "model": {
            "acc": 99.77558040644318,
            "cfm": [
                [
                    761468,
                    1
                ],
                [
                    1767,
                    24574
                ]
            ],
            "macc": 96.64584755258502,
            "miou": 96.52832080520561,
            "miou_class": {
                "0": "99.77",
                "1": "93.29"
            },
            "precision": 0.9999593082400814,
            "recall": 0.9329182643027979
        },
        "whole": {}
    },
    "PUNKTSKY_00004_1km_6106_492": {
        "model": {
            "acc": 99.97342534389041,
            "cfm": [
                [
                    2084050,
                    39
                ],
                [
                    516,
                    3851
                ]
            ],
            "macc": 94.09111838105903,
            "miou": 93.68845943867798,
      

In [11]:
with open(f"1abc_{model_name.split('.')[0]}_20_split.json", "w") as fp:
    json.dump(split_dict["test"],fp, cls=NumpyEncoder)

with open(f"1abc_{model_name.split('.')[0]}_20_mean.json", "w") as fp:
    json.dump(mean_metric_dict["test"],fp,cls=NumpyEncoder)

In [12]:
split_dict

{'test': {'PUNKTSKY_00004_1km_6106_492': {'model': {'acc': 99.97342534389041,
    'macc': 94.09111838105903,
    'miou': 93.68845943867798,
    'miou_class': {0: '99.97', 1: '87.40'},
    'precision': 0.989974293059126,
    'recall': 0.8818410808335242,
    'cfm': array([[2084050,      39],
           [    516,    3851]])},
   'whole': {}},
  'PUNKTSKY_00005_1km_6219_494': {'model': {'acc': 99.95956156359337,
    'macc': 96.86930962443239,
    'miou': 96.55755348279158,
    'miou_class': {0: '99.96', 1: '93.16'},
    'precision': 0.9933278223901748,
    'recall': 0.9374233812548758,
    'cfm': array([[3038439,     113],
           [   1123,   16823]])},
   'whole': {}},
  'PUNKTSKY_00004_1km_6106_510': {'model': {'acc': 99.98162876539651,
    'macc': 98.26249933550818,
    'miou': 98.21836421703632,
    'miou_class': {0: '99.98', 1: '96.46'},
    'precision': 0.9992467043314501,
    'recall': 0.9652537747862471,
    'cfm': array([[1055941,       4],
           [    191,    5306]])},
  

In [13]:
mean_metric_dict

{'test': {'acc': 99.92776199314608,
  'macc': 96.95661992822919,
  'miou': 96.27070086014045,
  'miou_0': 99.92571428571429,
  'miou_1': 92.61571428571428,
  'precision': 0.9853293859412974,
  'recall': 0.9392462451507156,
  'cfm': array([[12268942,     1790],
         [    6077,    97914]])}}